# import properties from different tables

In [ ]:
import os
import math
import glob
import pymysql
import pandas as pd
from sqlalchemy import create_engine

from tqdm.notebook import tqdm
from django.conf import settings
from appcreator import creator
from appcreator.import_utils import fetch_models, import_and_create_m2m, import_m2m_tables

In [ ]:
app_name = 'aschach'

In [ ]:
dbc = settings.LEGACY_DB_CONNECTION
sheet_id = "1dd9OlPFYCK1bHdC8U1eax13glEitlZhTDc-Cri_qA5A"

In [ ]:
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

In [ ]:
df = creator.gsheet_to_df(sheet_id)

In [ ]:
prop_row = df["value from"].str.contains('#Property') 

In [ ]:
custom_filter = df['field name technical'].str.contains('^maut_s')

In [ ]:
props = df[custom_filter]

In [ ]:
# props = df[prop_row]

In [ ]:
props

In [ ]:
model_dict = {x.__name__: x for x in fetch_models(app_name)}
for i, row in props.tail(1).iterrows():
    class_name = row['class name technical']
    cur_class = model_dict[class_name]
    try:
        prop_lit = row['value from'].split('#')[1]
    except IndexError:
        prop_lit = None
    table, prop_1, prop_2 = row['value from'].split('___')
    prop_2 = prop_2.split('#')[0]
    query = f"SELECT * FROM {table}"
    cur_model_attr = row['field name technical']
    field_type_source = row['field type']
    cur_class_nat_pk = cur_class.get_natural_primary_key()
    print(class_name, cur_class, cur_model_attr, table, prop_1, prop_2)
    data_source = pd.read_sql(query, con=db_connection)
    data_source = data_source[[prop_1, prop_2]].dropna()
    for ds_i, ds_row in tqdm(data_source.iterrows(), total=data_source.shape[0]):
        try:
            legacy_id_source = f"{float(ds_row[prop_1])}"
        except:
            continue
        try:
            curr_source = cur_class.objects.get(legacy_id=legacy_id_source)
        except Exception as e:
            curr_source = None
            continue
#             print(e)
#             print(f"no object of class {cur_class} with legacy_id {legacy_id_source} found")
        if curr_source is not None:
            my_val = ds_row[prop_2]

            if field_type_source.startswith('FK'):
                if cur_class_nat_pk == 'legacy_id':
                    my_val = f"{float(ds_row[prop_2])}"
                else:
                    my_val = ds_row[prop_2]
#                 print('ForeignKey')
                fk = cur_class._meta.get_field(cur_model_attr)
                rel_model_name = fk.related_model._meta.model_name
                kwargs = {cur_class_nat_pk: my_val}
                try:
                    temp_rel_obj = fk.related_model.objects.get(**kwargs)
                except Exception as e:
#                     print('###############')
#                     print(e)
#                     print(class_name, cur_class, cur_model_attr, table, prop_1, prop_2)
#                     print('###############')
                    temp_rel_obj = None
                if temp_rel_obj is not None:
                    setattr(curr_source, cur_model_attr, temp_rel_obj)
                    
            else:
                setattr(curr_source, cur_model_attr, my_val)
            curr_source.save()